In [1]:
import os
import cv2
import glob
import numpy as np


In [2]:
def get_label(img_path):
	# 讀取圖像
	image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

	img_class = 0 if "RI" in img_path else 1
	img_height, img_width = image.shape

	# 確保圖像是二值圖像，如果不是，則進行二值化處理
	_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

	# 在圖像周圍添加邊框（例如，增加10像素的邊框）
	border_size = 10
	binary_image_with_border = cv2.copyMakeBorder(binary_image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=[0])
	
	# 找到輪廓
	contours, _ = cv2.findContours(binary_image_with_border, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	contours = contours[0] - [border_size, border_size]


	max_x, max_y = np.max(contours, axis=0).squeeze()
	min_x, min_y = np.min(contours, axis=0).squeeze()

	# 調整輪廓坐標，去除邊框影響
	adjusted_contours = []
	for contour in contours:
		contour = contour.squeeze(0)
		contour = [contour[0] / img_width, contour[1] / img_height]
		adjusted_contours.append(contour)

	adjusted_contours.append(adjusted_contours[0])
	
	return {
		"class": img_class,
		"center_x": (max_x + min_x) / 2.0 / img_width,
		"center_y": (max_y + min_y) / 2.0 / img_height,
		"width": (max_x - min_x) / img_width,
		"height": (max_y - min_y) / img_height,
		"conture": adjusted_contours
	}



In [3]:
os.makedirs("./Dataset/Train_data", exist_ok=True)

In [4]:
for image_path in glob.glob("./Dataset/Train_data_label/label_img/*.png"):
	info = get_label(image_path)
	# copy file to Train_data
	os.system(f"cp ./Dataset/Train_data_label/img/{os.path.basename(image_path).replace('.png', '.jpg')} ./Dataset/Train_data/")

	with open(f"./Dataset/Train_data/{os.path.basename(image_path).replace('.png', '.txt')}", "w") as f:
		f.write(f"{info['class']} ")
		for point in info['conture']:
			f.write(f"{point[0]} {point[1]} ")